In [2]:
# 导入必要的库
# import mlir.all_passes_registration
import mlir.ir
import mlir.dialects.linalg.opdsl as opdsl
import numpy as np


In [3]:
# 定义矩阵乘法操作
def matmul(
    A=opdsl.TensorDef([8, 8], opdsl.F32),
    B=opdsl.TensorDef([8, 8], opdsl.F32),
    C=opdsl.TensorDef([8, 8], opdsl.F32),
):
    C[D.m, D.n] += A[D.m, D.k] * B[D.k, D.n]

# 初始化MLIR上下文
ctx = mlir.ir.Context()

# 定义输入和输出张量类型
input_type = mlir.ir.MemRefType.get_elemental_type(
    mlir.ir.F32Type.get(), 2, ["8", "8"]
)
output_type = mlir.ir.MemRefType.get_elemental_type(
    mlir.ir.F32Type.get(), 2, ["8", "8"]
)

# 创建MLIR module
with mlir.ir.InsertionPoint(ctx.module.body):
    # 创建矩阵乘法操作
    with mlir.ir.FuncOp.create(
        ctx,
        location=mlir.ir.Location.unknown(),
        name="matmul_linalg",
        type_=(
            mlir.ir.FunctionType.get(
                inputs=[input_type, input_type, output_type], results=[]
            )
        ),
    ) as func:
        # 将矩阵乘法操作添加到函数体中
        with mlir.ir.InsertionPoint(func.add_entry_block()):
            matmul(
                A=mlir.ir.Argument.create(
                    input_type, "A",
                ),
                B=mlir.ir.Argument.create(
                    input_type, "B",
                ),
                C=mlir.ir.Argument.create(
                    output_type, "C",
                ),
            )
            mlir.ir.ReturnOp.create(
                mlir.ir.Location.unknown(),
                operands=[],
            )

# 编译MLIR module
module = mlir.ir.Module.parse(str(ctx))

# 打印编译后的MLIR代码
print(module)

# 执行矩阵乘法操作
with mlir.ir.Context() as execution_context:
    # 创建输入和输出张量
    A = np.random.rand(8, 8).astype(np.float32)
    B = np.random.rand(8, 8).astype(np.float32)
    C = np.zeros((8, 8), dtype=np.float32)
    inputs = [mlir.ir.DenseElementsAttr.get(A), mlir.ir.DenseElementsAttr.get(B),]
    outputs = [mlir.ir.DenseElementsAttr.get(C)]

    # JIT编译和执行矩阵乘法操作
    jit_engine = mlir.execution_engine.ExecutionEngine(module)
    result = jit_engine.invoke("matmul_linalg", inputs, outputs)

    # 打印结果
    print(result)

AttributeError: module 'mlir.dialects.linalg.opdsl' has no attribute 'linalg'